In [1]:
#import dependencies
import os
import pandas as pd
import numpy as np

In [2]:
#specify path for schools file
schools_path= os.path.join("..","Resources","schools_complete.csv")
#specify path for students file
students_path= os.path.join("..","Resources", "students_complete.csv")

#read files:
schools_df = pd.read_csv(schools_path)
students_df = pd.read_csv(students_path)

#combine files:
all_data_df = pd.merge(schools_df, students_df, how="left", on=["school_name", "school_name"])
#assess column names
all_data_df.columns


Index(['School ID', 'school_name', 'type', 'size', 'budget', 'Student ID',
       'student_name', 'gender', 'grade', 'reading_score', 'math_score'],
      dtype='object')

In [3]:
#calculate total number of schools
num_schools=all_data_df["school_name"].nunique()
num_schools


15

In [4]:
#determine total number of students
num_students=all_data_df["Student ID"].count()
num_students

39170

In [5]:
#determine total budget
total_budget= (all_data_df["budget"].unique().sum())
total_budget

24649428

In [6]:
#determine average math score
mean_math= all_data_df["math_score"].mean()
mean_math

78.98537145774827

In [7]:
#determine average reading score
mean_read= all_data_df["reading_score"].mean()
mean_read

81.87784018381414

In [8]:
#determine percent passing in math and then in reading
#create the pass fail bins
bins=[0,69.9,100]
#create group names
group_names=["Fail", "Pass"]
#bin the data and make an output in a new column for both math summary and reading summary
all_data_df["math_summary"]=pd.cut(all_data_df["math_score"], bins, labels=group_names, include_lowest=True)
all_data_df["reading_summary"]=pd.cut(all_data_df["reading_score"], bins, labels=group_names)

In [9]:
#determine percent passing math
#create a subset of data with only the students that passed math
pass_math_df = all_data_df.loc[all_data_df["math_summary"]=="Pass"]
#detemine the number of students that passed math
num_pass_math =len(pass_math_df["math_summary"])
#determine the percent of students that passed math
percent_pass_math= num_pass_math/num_students*100
percent_pass_math

74.9808526933878

In [10]:
#determine percent passing in reading
#create a subset of data with only the students that passed reading
pass_read_df = all_data_df.loc[all_data_df["reading_summary"]=="Pass"]
#determine number of students that passed reading
num_pass_read= len(pass_read_df["reading_summary"])
#determine percent that passed reading
percent_pass_read = num_pass_read/num_students*100
percent_pass_read


85.80546336482001

In [11]:
#detemine percent passing both math and reading
#create a subset of data with only the students that passed both reading and math
pass_both_df= all_data_df.loc[(all_data_df["reading_summary"]=="Pass") & (all_data_df["math_summary"]=="Pass")]
#determine the number of students that passed both
num_pass_both=len(pass_both_df)
#determine the percent of students that passed both
percent_pass_both= num_pass_both/num_students*100
percent_pass_both

65.17232575950983

In [12]:
#create district summary dataframe:
district_data=[num_schools, num_students, total_budget, mean_math, mean_read, percent_pass_math, percent_pass_read, percent_pass_both]
district_summary_df= pd.DataFrame([district_data],\
    columns=("Total Schools","Total Students","Total Budget", "Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing")) 
#format Total Budget Column with $ and two decimal points
district_summary_df["Total Budget"]=district_summary_df["Total Budget"].astype("object").map('${:,.2f}'.format)
#format dataframe to align data to center
district_summary_df =district_summary_df.style.set_properties(**{'text-align': 'center'})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


In [13]:
#SCHOOL SUMMARY
#make a new dataframe grouped by school name:
grouped_df= all_data_df.groupby(['school_name'])
grouped_df.head()

#create new empty schools dataframe
schools_df=pd.DataFrame()

#Get Main Data for Summary Sheet
#determine school type
schools_df["Type"]= grouped_df['type'].first()
#determine total number of students at each school
schools_df["Total Students"]= grouped_df['Student ID'].count()
#determine total budget per school
schools_df["Total School Budget"]= grouped_df["budget"].first()
#determine per student budget
schools_df["Per Student Budget"]=(schools_df["Total School Budget"])/(schools_df["Total Students"])
#determine average math score
schools_df["Average Math Score"]= grouped_df["math_score"].mean()
#determine average reading score
schools_df["Average Reading Score"]= grouped_df["reading_score"].mean()

In [14]:
#determine percent of students at school passing math, reading and both:
#group passed math_df by school:
math_group_df=pass_math_df.groupby(["school_name"])
schools_df["% Passed Math"] = (math_group_df["math_summary"].count())/(schools_df["Total Students"])*100
#group passed_reading_df by school
reading_group_df=pass_read_df.groupby(["school_name"])
schools_df["% Passed Reading"] = (reading_group_df["reading_summary"].count())/(schools_df["Total Students"])*100
#group passed both by school
both_group_df=pass_both_df.groupby(["school_name"])
schools_df["% Passed Both"]= (both_group_df["Student ID"].count())/(schools_df["Total Students"])*100

In [15]:
#format the Schools Summary DataFrame
#format entire df as string
schools_df_formatted=schools_df.astype('object')
schools_df_formatted["Total School Budget"]=schools_df_formatted["Total School Budget"].map('${:,.2f}'.format)
schools_df_formatted["Per Student Budget"]=schools_df_formatted["Per Student Budget"].map('${:,.2f}'.format)

schools_df_formatted

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Both
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.0484,81.034,66.6801,81.9333,54.6423
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.0619,83.9758,94.1335,97.0398,91.3348
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7118,81.158,65.9885,80.7392,53.2045
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1026,80.7463,68.3096,79.299,54.2899
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.3515,83.8168,93.3924,97.139,90.5995
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.2898,80.9344,66.753,80.863,53.5275
Holden High School,Charter,427,"$248,087.00",$581.00,83.8033,83.815,92.5059,96.2529,89.2272
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6294,81.1827,65.6839,81.3164,53.5139
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.0725,80.9664,66.0576,81.2224,53.5392


In [16]:
#Top 5 Performing Schools: 
schools_sort_top = schools_df_formatted.sort_values('% Passed Both',ascending=False)
top_5_df=schools_sort_top.head(5)
top_5_df

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Both
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.0619,83.9758,94.1335,97.0398,91.3348
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.4183,83.8489,93.2722,97.3089,90.948
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.3515,83.8168,93.3924,97.139,90.5995
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.2742,83.9895,93.8677,96.5396,90.5826
Pena High School,Charter,962,"$585,858.00",$609.00,83.8399,84.0447,94.5946,95.9459,90.5405


In [17]:
#Bottom 5 Performing Schools: 
schools_sort_bottom = schools_df_formatted.sort_values('% Passed Both',ascending=True)
bottom_5_df=schools_sort_bottom.head(5)
bottom_5_df

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Both
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8427,80.7447,66.3666,80.2201,52.9882
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7118,81.158,65.9885,80.7392,53.2045
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6294,81.1827,65.6839,81.3164,53.5139
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.2898,80.9344,66.753,80.863,53.5275
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.0725,80.9664,66.0576,81.2224,53.5392


In [18]:
#Math (and Reading) Scores By Grade 
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
# Create a pandas series for each grade. Hint: use a conditional statement.
# Group each series by school
# Combine the series into a dataframe
# Optional: give the displayed data cleaner formatting

In [19]:
#filter data to grade 9
grade9_series= all_data_df.loc[all_data_df['grade']=='9th',:]
#group by school name
grade9_grouped=grade9_series.groupby('school_name')
#determine mean reading score
grade9_avg_reading=grade9_grouped['reading_score'].mean()
#determine mean math score
grade9_avg_math=grade9_grouped['math_score'].mean()

In [20]:
#filter data to grade 10
grade10_series= all_data_df.loc[all_data_df['grade']=='10th',:]
#group by school
grade10_grouped=grade10_series.groupby('school_name')
#determine mean reading score
grade10_avg_reading=grade10_grouped['reading_score'].mean()
#determine mean math score
grade10_avg_math=grade10_grouped['math_score'].mean()

In [21]:
#filter to grade 11
grade11_series= all_data_df.loc[all_data_df['grade']=='11th',:]
#group by school
grade11_grouped=grade11_series.groupby('school_name')
#mean reading score
grade11_avg_reading=grade11_grouped['reading_score'].mean()
#mean math score
grade11_avg_math=grade11_grouped['math_score'].mean()

In [22]:
#filter to grade 12
grade12_series= all_data_df.loc[all_data_df['grade']=='12th',:]
#group by school
grade12_grouped=grade12_series.groupby('school_name')
#mean reading score
grade12_avg_reading=grade12_grouped['reading_score'].mean()
#mean math score
grade12_avg_math=grade12_grouped['math_score'].mean()

In [23]:
# create avg math score dataframe
avg_math_by_grade_df=pd.DataFrame({'9th':grade9_avg_math, '10th':grade10_avg_math,'11th':grade11_avg_math,'12th':grade12_avg_math})
avg_math_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [24]:
# create avg reading score dataframe
avg_reading_by_grade_df=pd.DataFrame({'9th':grade9_avg_reading, '10th':grade10_avg_reading,'11th':grade11_avg_reading,'12th':grade12_avg_reading})
avg_reading_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [25]:
#score schools by spending
#create the bins
spend_bins=[0,585, 630, 645, 675]
#create group names
group_names=["<$584", "$585-629", "$630-644", "$645-675"]
#bin the data and make an output in a new column for both math summary and reading summary
schools_df["Spending Ranges Per Student"]=pd.cut(schools_df["Per Student Budget"], spend_bins, labels=group_names, include_lowest=True)
#group by spend per student and get the mean for each dataset.
spending_grouped_df=schools_df.groupby('Spending Ranges Per Student').mean()
#remove ancillary data columns
spending_grouped_df= spending_grouped_df.drop(['Total Students','Total School Budget', 'Per Student Budget'], axis=1)
spending_grouped_df

,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Both
Spending Ranges Per Student,,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-644,78.518855,81.624473,73.484209,84.391793,62.857656
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


In [26]:
#score schools by size
#create the bins
size_bins=[0,1000, 2000, 5000]
#create group names
group_names=["Small, <1000 Students", "Medium, 1000-2000 Students", "Large, 2000-5000 Students"]
#bin the data and make an output in a new column for both math summary and reading summary
schools_df["School Size"]=pd.cut(schools_df["Total Students"], size_bins, labels=group_names, include_lowest=True)
#group by spend per student and get the mean for each dataset.
size_grouped_df=schools_df.groupby('School Size').mean()
#remove ancillary data columns
size_grouped_df= size_grouped_df.drop(['Total Students','Total School Budget', 'Per Student Budget'], axis=1)
size_grouped_df

,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Both
School Size,,,,,
"Small, <1000 Students",83.821598,83.929843,93.550225,96.099437,89.883853
"Medium, 1000-2000 Students",83.374684,83.864438,93.599695,96.790680,90.621535
"Large, 2000-5000 Students",77.746417,81.344493,69.963361,82.766634,58.286003


In [27]:
#scores by school type
type_grouped_df=schools_df.groupby('Type').mean()
#remove ancillary data columns
type_grouped_df= type_grouped_df.drop(['Total Students','Total School Budget', 'Per Student Budget'], axis=1)
type_grouped_df

,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Both
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
